# NLP - Prova 1

Nesta prova, utilizaremos um dataset fornecido por uma empresa de e-commerce. Trata-se de um dataset que contém reviews de pedidos feitos na Internet ou através do aplicativo. Cada review tem uma nota (*score*) que vai de 1 (muito ruim) a 5 (muito bom). Também, cada review tem uma mensagem deixada pelo cliente, que pode esclarecer o que levou à atribuição da nota. O dataset é o seguinte:

In [7]:
import pandas as pd 
df = pd.read_csv('reviews.csv')
df.head()

,review_id,review_score,review_comment_message
0,e64fb393e7b32834bb789ff8bb30750e,5,Recebi bem antes do prazo estipulado.
1,f7c4243c7fe1938f181bec41a392bdeb,5,Parabéns lojas lannister adorei comprar pela I...
2,8670d52e15e00043ae7de4c01cc2fe06,4,aparelho eficiente. no site a marca do aparelh...
3,4b49719c8a200003f700d3d986ea1a19,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,3948b09f7c818e2d86c9a546758b2335,5,"Vendedor confiável, produto ok e entrega antes..."


A empresa, porém, gostaria de saber quais são os aspectos em que pode melhorar. Para fazer isso, jogou todos os reviews em um LLM (o procedimento para isso está em `consultar_llm.py`, que acompanha este notebook), e então coletou as respostas. Em sua execução, o LLM retornou o seguinte:

## RESPOSTA DO LLM

Para identificar os elementos mais elogiados e reclamados, vou analisar as avaliações buscando palavras-chave e temas recorrentes.

**Elogios Mais Comuns:**

*   **Entrega Rápida/No Prazo:** Este é o elogio mais frequente. Termos como "chegou antes do prazo", "entrega rápida", "no prazo" aparecem repetidamente.
*   **Produto Bom/Excelente Qualidade:** Muitos clientes expressam satisfação com a qualidade do produto em si. Palavras como "excelente", "bom", "perfeito", "recomendo", "atendeu minhas expectativas", "cumpre o que promete" são comuns.
*   **Bom Atendimento/Comunicação:** Alguns clientes elogiam a comunicação clara e eficiente com o vendedor.
*   **Embalagem:** Alguns clientes elogiaram a forma como o produto foi embalado.

**Reclamações Mais Comuns:**

*   **Não Recebimento do Produto:** Esta é a reclamação mais grave e frequente. Várias avaliações mencionam "não recebi o produto" ou "ainda não recebi".
*   **Atraso na Entrega:** Mesmo quando o produto é entregue, o atraso é uma reclamação comum.
*   **Produto com Defeito/Danificado:** Alguns clientes relatam receber produtos com defeito, amassados ou com peças faltando.
*   **Produto Errado:** Alguns clientes relatam ter recebido o produto errado.
*   **Problemas com a Compra/Cancelamento:** Alguns clientes relatam problemas com o cancelamento da compra.
*   **Problemas com a Comunicação:** Alguns clientes reclamam da dificuldade em se comunicar com o vendedor após a compra ou da falta de resposta às suas dúvidas.
*   **Voltagem Errada:** Um cliente reclamou que o produto veio com a voltagem errada.
*   **Qualidade do Produto Abaixo do Esperado:** Alguns clientes expressam insatisfação com a qualidade do produto, mencionando material ruim, acabamento imperfeito ou falsificação.
*   **Problemas com a Entrega pelos Correios:** Um cliente reclamou do serviço dos correios.

**Resumo:**

*   **Elogios:** Entrega rápida/no prazo e boa qualidade do produto são os pontos fortes.
*   **Reclamações:** Problemas com a entrega (não recebimento, atraso) e produtos com defeito/qualidade inferior são os principais pontos fracos.

**Recomendações:**

A empresa deve priorizar a melhoria dos processos de entrega para garantir que os produtos cheguem aos clientes no prazo e em perfeitas condições. Além disso, deve investir no controle de qualidade dos produtos e na comunicação com os clientes para resolver problemas de forma rápida e eficiente.


## Discussão

Mesmo que o LLM possa dar pistas importantes para o desenvolvimento desta solução, há um problemaa fundamental em simplesmente "acreditar" nessa resposta: ela não tem base em dados nem em uma metodologia científica, então é impossível criticá-la, replicá-la ou saber de suas limitações.

A tarefa neste exercício é encontrar dados, usando metodologias clássicas de NLP, que corroborem ou que contradigam a conclusão trazida pelo LLM.


# Exercícios

## Exercício 1

Faça e avalie (usando uma métrica) um modelo preditivo baseado no modelo Bag-of-Words capaz de prever, à partir do texto do review, se ele será positivo (notas 4 ou 5) ou negativo (notas 1, 2 ou 3). Use estratégias adequadas para mostrar que o modelo não está enviesado (por exemplo, retornando sempre 'positivo' para o review). Justifique todas as suas decisões (tipo de vetorizador, tipo de classificador, métrica(s) usada(s) para avaliação) em comentários no código.

Dica: `df['positivo'] = df['review_score'].apply(lambda x: x > 3)`

In [42]:
#exercicio1
#não apague o comentário acima!!!
# Faça todo o seu código (e também os comentários) nesta célula!

# Primeiramente criaremos uma coluna para nosso dataset que será nosso Y, basicamente expressando 1 (True) caso o review seja positivo e 0 
# (False) caso o review seja negativo

df['positivo'] = df['review_score'].apply(lambda x: x > 3)
total_reviews_positivos = df[df['positivo'] == True]['positivo'].count()
total_reviews_negativos = df[df['positivo'] == False]['positivo'].count()

print(f'Total de revies positivos: {total_reviews_positivos} | % de reviews positivos: {total_reviews_positivos / (total_reviews_positivos + total_reviews_negativos) * 100:.2f}%')
print(f'Total de revies negativos: {total_reviews_negativos} | % de reviews negativos: {total_reviews_negativos / (total_reviews_positivos + total_reviews_negativos) * 100:.2f}%\n')

# Carregando arquivo de stopwords em portugues, link: https://gist.github.com/alopes/5358189
stopwords = open("stopwords.txt", "r") 
stopwords_data = stopwords.read() 
data_into_list = stopwords_data.strip()
data_into_list = data_into_list.replace(' ','').replace('\n', '.').split(".") 
stopwords.close() 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Criando a pipeline que contem nosso vetorizador e nosso classificador

model = Pipeline([
    ('vectorizer', CountVectorizer(binary=True, stop_words=data_into_list, min_df=3, max_df=0.8)),
    ('classifier', BernoulliNB())
])

# Vetorizador: para o vetorizador vamos usar o CountVectorizer do sklearn, que tem como função converter uma coleção de textos em uma matriz de contagem dos 
# nossos tokens. Como nosso classificador é baseado na distribuição de Bernoulli, modelo binario, que considera apenas a presença ou ausência de uma feature 
# setamos binary=True para que cada palavra no documento seja transofrmada em 1 ou 0, indicando a presença ou ausência da palavra.
# Além disso consideramos e removemos as stopwords em português da lista de tokens para evitar sua influência sobre o classificador. 
# Por fim no vetorizador também configuramos os parâmetros min_df e max_df aonde p valor absoluto ou fracionario representa, a frequência mínima/máxima de um termo,
# neste caso colocamos o valor absoluto 3 como min_df, aonde basicamente descartamos palavras com 3 ou menos aparições e max_df palavras com uma taxa de aparição
# maior que 80% nos textos, isso ajuda com problemas futuros de outliers e evita que nosso classificador lide com o tão temido "overfitting".

# Classificador:
# O modelo escolhido é BernoulliNB é construido em cima da distribuição de Bernoulli, que basicamente classifica em resultados binarios baseado na presença 
# ou ausência de features, seguindo nesse caso a idéia de modelo Bag-of-Words. A idéia é basicamente que o classificador recebe o vetor mencionado acima que 
# descreve o conteudo do dataset (nesse caso os tokens), e então o classificador usa esse vetor x que representa a probabilidade que aquele item (palavra) 
# esteja presente e logo corresponde a cada uma das classes conhecidas, e a partir da probabilidade das mesmas classifica então nesse caso o review como 
# positivo ou negativo (baseado nas palavras/tokens presentes)

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df['review_comment_message'], df['positivo'], test_size=0.2)

# Treinar a pipeline 
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo: {accuracy:.2f}%\n')

# Como pode ser observado no print da célula acima, obtivemos uma acurácia de 85%, agora iremos avaliar o resultado do 
# modelo preditivo para garantir que o mesmo não esteja eviesado.

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# Utilizamos o classification_report do sklearn.metrics para tirar algumas métricas do nosso modelo, nesse caso temos o valor absoluto chutado por ele para cada 
# possível resultado, nesse caso ele considerou 2894 avaliações como negativas e 5302 como positivas algo que parece fazer sentido até considerando nosso dataset 
# que como mostrado acima possui uma taxa de aproximadamente 65% de reviews positivos e 35% de reviews negativos. Outro ponto importante é que ele não está chutando
# apenas uma única resposta. 
# Outra métrica interessante é considerarmos o precision e recall para cada um dos casos: 
# Quando olhamos para o precision 0.84 e recall 0.74 dos reviews negativos e precision 0.86 e recall 0.92 dos reviews positivos podemos inferir que nosso modelo
# é menos confiante quando classificando os reviews negativos ao contrário dos positivos, por isso a diferença de recall entre os mesmos, 
# entretanto a precisão é bem semelhante entre ambos, enfatizando em uma taxa de acerto proporcional para ambos os casos. 

Total de revies positivos: 26530 | % de reviews positivos: 64.74%
Total de revies negativos: 14447 | % de reviews negativos: 35.26%

Acurácia do modelo: 0.86%

              precision    recall  f1-score   support

       False       0.83      0.75      0.79      2900
        True       0.87      0.92      0.89      5296

    accuracy                           0.86      8196
   macro avg       0.85      0.83      0.84      8196
weighted avg       0.86      0.86      0.86      8196



## Exercício 2

Analise seu modelo e encontre quais são as palavras e/ou n-gramas que são melhores preditoras do número de estrelas (alto ou baixo) recebido um review (dica: você pode usar `pipeline[:-1].get_feature_names_out()` para encontrar uma lista com todos os tokens do seu vocabulário). Ao usar elementos do sklearn, use um comentário no código para explicar o que esses elementos significam e como eles podem apontar para o poder preditivo de cada palavra (ou n-grama). Por fim, use comentários no código para explicar como suas descobertas corroboram, contradizem, ou relativizam o que foi proposto pelo LLM.

In [68]:
#exercicio2
#não apague o comentário acima!!!
# Faça todo o seu código (e também os comentários) nesta célula!

# A linha abaixo encontra uma lista com todos os tokens do nosso vocabulário
tokens = model[:-1].get_feature_names_out()
lista_tokens = []

# A idéia é basicamente ir inteirando por todos os tokens e utlizando a função predict_proba do sklearn que retorna a probabilidade do token pertencer a classe
# dada suas características e assim conseguimos inferir que aqueles que apresentarem a maior diferença entre as probabilidades são as palavras melhores preditoras
# de um review bom (número de estrelas altos) e aquele que apresentar uma menor diferença entre as probabilidade 
# o melhor preditor de uma review ruim (número de estrelas baixos)


for token in tokens:
    proba = model.predict_proba([token])[0]
    if proba[0] > proba[1]:
        lista_tokens.append((token,proba[0]))
    else:
        lista_tokens.append((token,proba[1]))


sorted_by_second = sorted(lista_tokens, key=lambda tup: tup[1])

# Abaixo pegamos a lista e olhamos para os primeiros 5 e ultimos 5 elementos da lista ordenada baseada no valor encontrado e então separamos apenas a palavra
# formando assim duas listas, uma com as palavras para reviews positivas e outra para negativas.
top5_words_negative = [i[0] for i in sorted_by_second[0:5]]
top5_words_positive = [i[0] for i in list(reversed(sorted_by_second))[0:5]]

print(f'Melhores palavras preditoras de reviews negativos e sua probabilidade: {sorted_by_second[0:5]}')
print(f'Melhores palavras preditoras de reviews positivas e sua probabilidade: {list(reversed(sorted_by_second))[0:5]}')

print(f'Melhores palavras preditoras de reviews negativos: {top5_words_negative}')
print(f'Melhores palavras preditoras de reviews positivas: {top5_words_positive}')

Melhores palavras preditoras de reviews negativos e sua probabilidade: [('pessima', np.float64(0.503474857199898)), ('ressarcimento', np.float64(0.503474857199898)), ('tento', np.float64(0.503474857199898)), ('estorno', np.float64(0.5042114005393379)), ('lamentável', np.float64(0.5158034729090769))]
Melhores palavras preditoras de reviews positivas e sua probabilidade: [('show', np.float64(0.9994476839700426)), ('amei', np.float64(0.9993417862561389)), ('adorei', np.float64(0.9992589020116213)), ('parabéns', np.float64(0.9992163228310875)), ('excelente', np.float64(0.9991388104959608))]
Melhores palavras preditoras de reviews negativos: ['pessima', 'ressarcimento', 'tento', 'estorno', 'lamentável']
Melhores palavras preditoras de reviews positivas: ['show', 'amei', 'adorei', 'parabéns', 'excelente']


# Rubricas e auto-avaliação assistida por IA

Este teste tem duas questões. Porém, são questões bastante complexas. Para verificar se você não esqueceu de nada em nenhuma questão, você pode usar o script `autoavaliacao.py`.

Cada um dos ítens cumpridos corretamente nas rubricas disponíveis em `autoavaliacao.py` terão o mesmo valor nas questões (1/8 na questão 1, 1/4 na questão 2). Um ítem cumprido parcialmente vale metade dos pontos. Um ítem feito errado não vale pontos. Cada uma das questões tem o mesmo valor (50% do total).

**IMPORTANTE**: o fato de o LLM dizer que um ítem da rubrica foi cumprido, não significa que ele **realmente** foi cumprido. O LLM avalia apenas se o ítem **existe**. O LLM é uma **ferramenta** e a sua utilidade depende do **seu** julgamento crítico.

Após o término da prova, salve seu notebook e entregue no Blackboard.

## Avaliação da prova

Se desejar, comente os pontos abaixo:

**Experiência da prova**: ao fazer esta prova, me senti...

**Dificuldades técnicas ou de compreensão do enunciado**: ...

**O script de auto-avaliação me ajudou**: sim, não, em partes...?